In [2]:
!pip install deap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for deap: filename=deap-1.4.1-cp311-cp311-macosx_11_0_arm64.whl size=103874 sha256=9b050cdee6ed42fa03f98c7ff0a33d6c15aee0635a25b67235bc22d9d1df5c75
  Stored in directory: /Users/amansharma/Library/Caches/pip/wheels/f8/64/b8/65eacfbff3024ae2e2beb22e691d5c8abb89fbd863b8049b5f
Successfully built deap


In [7]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sympy.combinatorics.graycode import bin_to_gray,gray_to_bin
from keras.models import load_model
import operator
import random
import numpy
import math
from deap import base
from deap import creator
from deap import tools
from sklearn.metrics import accuracy_score
import math
from keras import datasets

In [8]:
model = load_model('Cifar10Model1.h5')

In [9]:
(X_train,y_train),(X_test,y_test) = datasets.cifar10.load_data()

In [10]:
def IP2bit(b):
    newInd_IP=[]
    for index in range(len(b)):
        chromosome = format(b[index], "016b")
        chromosome = bin_to_gray(chromosome)
        newInd_IP.extend(chromosome)
    #newInd_IP = bin_to_gray(newInd_IP)
    newInd_IP = [int(i) for i in newInd_IP]
    return newInd_IP

def chrom2real(c):
    numOfBits = 16
    indasstring=''.join(map(str, c))
    degray=gray_to_bin(indasstring)
    numasint=int(degray, 2) # convert to int from base 2 list
    numinrange=-1+2.0*numasint/(2**numOfBits)
    return numinrange

def separatevariables(v):
#     print(v)
    numOfBits = 16
    sep = []
    for i in range (0,numOfBits*650,numOfBits):
        sep.append(chrom2real(v[i:i+numOfBits]))
    return sep

In [12]:
# Sample code for PSO, using DEAP library

numOfBits = 16
Chrom_length = 650
bit = 16
posMinInit      = 1
posMaxInit      = 2**bit
VMaxInit        = 0.1*posMaxInit
VMinInit        = -0.1*posMaxInit
populationSize  = 50
dimension       = int((Chrom_length*numOfBits)/bit)n
interval        = 1
iterations      = 200
xmax = 50
best_value = []
fitness_list_best = []
fitness_list_worst = []
r = 0.018
#Parameter setup

wmax = 0.9 #weighting
wmin = 0.4 
c1   = 2.0
c2   = 2.0

creator.create("FitnessMin", base.Fitness, weights=(1.0,)) # -1 is for minimise
creator.create("Particle", list, fitness=creator.FitnessMin, speed=list, smin=None, smax=None, best=None)
# particle rerpresented by list of 5 things
# 1. fitness of the particle, 
# 2. speed of the particle which is also going to be a list, 
# 3.4. limit of the speed value, 
# 5. best state the particle has been in so far.

def generate(size, smin, smax):
    part = creator.Particle(int(random.uniform(posMinInit, posMaxInit)) for _ in range(size)) 
    part.speed = [random.uniform(VMinInit, VMaxInit) for _ in range(size)]
    part.smin = smin 
    part.smax = smax
    return part

def updateParticle(part, best, weight,pop):

    r1 = (random.uniform(0, 1) for _ in range(len(part)))
    r2 = (random.uniform(0, 1) for _ in range(len(part)))

    v_r0 = [weight*x for x in part.speed]
    v_r1 = [c1*x for x in map(operator.mul, r1, map(operator.sub, part.best, part))] # local best
    v_r2 = [c2*x for x in map(operator.mul, r2, map(operator.sub, best, part))] # global best
    
    part.speed = [0.7*x for x in map(operator.add, v_r0, map(operator.add, v_r1, v_r2))]

    part[:] = list(map(operator.add, part, part.speed))
    part[:] = [1 if i<1 else i for i in part[:]]
    part[:] = [posMaxInit if i>posMaxInit else i for i in part[:]]
    part[:] = [int(i) for i in part[:]]

def loss_func(part):
    new_pop = IP2bit(part)
    layer_weights = separatevariables(new_pop)

    weights = np.array(layer_weights[0:640]).reshape(64,10)
    bias = np.array(layer_weights[640:]).reshape(10,)
    layer_weights = [weights,bias]
    model.layers[-1].set_weights(layer_weights)

    y_pred = model.predict(X_train,verbose = 0)
    y_pred_classes = [np.argmax(element) for element in y_pred]
    loss = accuracy_score(y_train,y_pred_classes)
    return loss,

toolbox = base.Toolbox()
toolbox.register("particle", generate, size=dimension, smin=-0.1*posMaxInit, smax=0.1*posMaxInit)
toolbox.register("population", tools.initRepeat, list, toolbox.particle)
toolbox.register("update", updateParticle)
toolbox.register("evaluate", loss_func) 

def main():
    pop = toolbox.population(n=populationSize) # Population Size

    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register("std", numpy.std)
    stats.register("min", numpy.min)
    stats.register("max", numpy.max)

    logbook = tools.Logbook()
    logbook.header = ["gen", "evals"] + stats.fields

    best = None

    #begin main loop
    for g in range(iterations):
        w = wmax - (wmax-wmin)*g/iterations #decaying inertia weight
        for part in pop:
            part.fitness.values = toolbox.evaluate(part) #actually only one fitness value
        
            #update local best
            if (not part.best) or (part.best.fitness < part.fitness):   
            #   best is None   or  current value is better              #< is overloaded        
                part.best = creator.Particle(part)
                part.best.fitness.values = part.fitness.values
            
            #update global best
            if (not best) or best.fitness < part.fitness:
                best = creator.Particle(part)
                best.fitness.values = part.fitness.values
                print(best.fitness.values[0])
        for part in pop:
            toolbox.update(part, best,w,pop)

        sorted_pop = sorted(pop, key=lambda ind: ind.fitness.values, reverse=True)
        fitness_list_best.append(sorted_pop[0].fitness.values[0])
        fitness_list_worst.append(sorted_pop[-1].fitness.values[0])
        # Gather all the fitnesses in one list and print the stats

        if g%interval==0: # interval 
            logbook.record(gen=g, evals=len(pop), **stats.compile(pop))
            print(logbook.stream)
            #print('best ',best, best.fitness)
    
    print('best particle position is ',best)
    print(best.fitness)
    best_value.extend(best)
    return pop, logbook, best,best_value,fitness_list_best,fitness_list_worst

if __name__ == "__main__":
    main()


/opt/anaconda3/envs/ds/lib/python3.11/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/opt/anaconda3/envs/ds/lib/python3.11/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'Particle' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
2024-11-21 12:17:38.159764: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


0.09972
0.14214
gen	evals	avg      	std      	min    	max    
0  	50   	0.0999492	0.0223573	0.03794	0.14214
0.14662
1  	50   	0.101554 	0.0163641	0.0597 	0.14662
0.18062
2  	50   	0.100147 	0.0218435	0.05072	0.18062
3  	50   	0.0980096	0.0228445	0.04742	0.1646 
4  	50   	0.100444 	0.0221582	0.04168	0.15174
5  	50   	0.0992772	0.016056 	0.05692	0.14054
6  	50   	0.102634 	0.0175284	0.0672 	0.15052
7  	50   	0.0991756	0.0196444	0.04146	0.15006
8  	50   	0.100094 	0.024872 	0.0471 	0.1582 
9  	50   	0.0941192	0.0195146	0.04342	0.1403 
10 	50   	0.0977524	0.0219839	0.03206	0.17026
11 	50   	0.0965684	0.019424 	0.04378	0.14526
12 	50   	0.095478 	0.0184868	0.0499 	0.1395 
13 	50   	0.0946944	0.0143612	0.06348	0.13346
14 	50   	0.102168 	0.0161696	0.0605 	0.1383 
15 	50   	0.0926028	0.0206951	0.04704	0.1392 
16 	50   	0.0971976	0.0205254	0.04614	0.1581 
17 	50   	0.0984892	0.0204452	0.0487 	0.1638 
18 	50   	0.0988632	0.0207239	0.02928	0.14922
19 	50   	0.0962708	0.0184201	0.0387 	0.14294
20

In [14]:
new_pop = IP2bit(best_value)
layer_weights = separatevariables(new_pop)
weights = np.array(layer_weights[0:640]).reshape(64,10)
bias = np.array(layer_weights[640:]).reshape(10,)
layer_weights = [weights,bias]
model.layers[-1].set_weights(layer_weights)

y_pred = model.predict(X_test,verbose = 0)
y_pred_classes = [np.argmax(element) for element in y_pred]
loss = accuracy_score(y_test,y_pred_classes)
loss

0.2053

In [16]:
fitness_list_best

[0.14214,
 0.14662,
 0.18062,
 0.1646,
 0.15174,
 0.14054,
 0.15052,
 0.15006,
 0.1582,
 0.1403,
 0.17026,
 0.14526,
 0.1395,
 0.13346,
 0.1383,
 0.1392,
 0.1581,
 0.1638,
 0.14922,
 0.14294,
 0.16014,
 0.13098,
 0.15734,
 0.15978,
 0.13862,
 0.19632,
 0.1405,
 0.15296,
 0.17464,
 0.15922,
 0.16714,
 0.17848,
 0.14708,
 0.14736,
 0.17354,
 0.14392,
 0.18106,
 0.14798,
 0.16372,
 0.16164,
 0.18952,
 0.1407,
 0.12446,
 0.16218,
 0.13792,
 0.14276,
 0.1688,
 0.17418,
 0.14536,
 0.13248,
 0.16142,
 0.1479,
 0.15428,
 0.12814,
 0.1665,
 0.15724,
 0.15594,
 0.15658,
 0.16544,
 0.1493,
 0.15432,
 0.14576,
 0.14832,
 0.14054,
 0.14246,
 0.20478,
 0.16328,
 0.17044,
 0.19378,
 0.1938,
 0.17016,
 0.19384,
 0.20238,
 0.19934,
 0.19434,
 0.18718,
 0.20292,
 0.1841,
 0.20676,
 0.20584,
 0.20724,
 0.20274,
 0.20274,
 0.2056,
 0.2074,
 0.20312,
 0.20724,
 0.20724,
 0.20754,
 0.20762,
 0.20762,
 0.20762,
 0.20762,
 0.20762,
 0.20762,
 0.20762,
 0.20762,
 0.20762,
 0.20762,
 0.20762,
 0.20762,
 0.20762

In [17]:
fitness_list_worst

[0.03794,
 0.0597,
 0.05072,
 0.04742,
 0.04168,
 0.05692,
 0.0672,
 0.04146,
 0.0471,
 0.04342,
 0.03206,
 0.04378,
 0.0499,
 0.06348,
 0.0605,
 0.04704,
 0.04614,
 0.0487,
 0.02928,
 0.0387,
 0.02544,
 0.05664,
 0.05378,
 0.04916,
 0.05976,
 0.05786,
 0.0202,
 0.0594,
 0.03338,
 0.04776,
 0.02222,
 0.051,
 0.0363,
 0.0241,
 0.05196,
 0.01338,
 0.04158,
 0.05868,
 0.05204,
 0.04512,
 0.05216,
 0.04538,
 0.03134,
 0.0442,
 0.03752,
 0.0442,
 0.04348,
 0.03712,
 0.06002,
 0.05576,
 0.05394,
 0.05056,
 0.02662,
 0.04802,
 0.03182,
 0.0499,
 0.05414,
 0.04022,
 0.04376,
 0.0468,
 0.0441,
 0.04786,
 0.04866,
 0.05352,
 0.0501,
 0.04454,
 0.0348,
 0.05096,
 0.05394,
 0.03748,
 0.04434,
 0.05504,
 0.02448,
 0.04516,
 0.03356,
 0.03516,
 0.06098,
 0.02636,
 0.0433,
 0.05018,
 0.02576,
 0.0295,
 0.04816,
 0.054,
 0.04732,
 0.06434,
 0.05978,
 0.054,
 0.04542,
 0.0326,
 0.06282,
 0.04294,
 0.02538,
 0.05846,
 0.05638,
 0.05848,
 0.03962,
 0.04912,
 0.05458,
 0.04892,
 0.05078,
 0.0344,
 0.03406

In [19]:
file_name = "fitness_list_best.txt"

# Saving the list to a text file
with open(file_name, "w") as file:
    for item in fitness_list_best:
        file.write(f"{item}\n")

print(f"List has been saved to {file_name}")

List has been saved to fitness_list_best.txt


In [20]:
file_name = "fitness_list_worst.txt"

# Saving the list to a text file
with open(file_name, "w") as file:
    for item in fitness_list_worst:
        file.write(f"{item}\n")

print(f"List has been saved to {file_name}")

List has been saved to fitness_list_worst.txt
